In [1]:
# Imports
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from city_sustainability.quality import life_quality
from city_sustainability.models import unet_model
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.preprocessing import image_resize
from city_sustainability.loading_data_400x400 import image_and_label_arrays_400
from city_sustainability.loading_data_400x400 import loading_paths_400

2023-05-25 23:04:00.690099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 23:04:00.714875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 23:04:00.715659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 23:04:01.265891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Retrieve image_paths and label_paths
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths_400, label_paths_400 = loading_paths_400(data_path_1)

In [3]:
# Check length of image_paths
len(image_paths_400)

2687

In [4]:
# Check length of label_paths
len(label_paths_400)

2687

In [6]:
# Import all image array as X and label array as y
X, y = image_and_label_arrays_400(image_paths_400, label_paths_400, sampling_ratio=0.33)

In [7]:
# Apply holdout
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [8]:
# Scale X and y
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [9]:
# Confirm shapes
X_train.shape
X_test.shape
y_train.shape
y_test.shape
X_train_scaled.shape
X_test_scaled.shape

(267, 400, 400, 3)

In [10]:
X_test.shape

(267, 400, 400, 3)

In [11]:
y_train.shape

(620, 400, 400, 9)

In [12]:
y_test.shape

(267, 400, 400, 9)

In [13]:
X_train_scaled.shape

(620, 400, 400, 3)

In [14]:
X_test_scaled.shape

(267, 400, 400, 3)

In [16]:
# Create the model
model = unet_model.build_vgg16_model(input_shape=(400, 400, 3), num_classes = 9)

In [17]:
# Compile the model

unet_model.compile_model(model)

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# callbacks=[early_stopping])

In [18]:
# Train the model

unet_model.train_model(model, X_train_scaled, y_train, epochs = 50, batch_size=16, validation_split=0.2)

Epoch 1/50
31/31 [==============================] - 341s 11s/step - loss: 2.0995 - accuracy: 0.3094 - compute_iou: 0.1309 - val_loss: 3.6931 - val_accuracy: 0.3587 - val_compute_iou: 0.2142 - lr: 1.0000e-04
Epoch 2/50
31/31 [==============================] - 336s 11s/step - loss: 1.6263 - accuracy: 0.4428 - compute_iou: 0.1882 - val_loss: 2.7614 - val_accuracy: 0.3746 - val_compute_iou: 0.2254 - lr: 1.0000e-04
Epoch 3/50
31/31 [==============================] - 336s 11s/step - loss: 1.5024 - accuracy: 0.4785 - compute_iou: 0.2113 - val_loss: 2.0967 - val_accuracy: 0.4108 - val_compute_iou: 0.2419 - lr: 1.0000e-04
Epoch 4/50
31/31 [==============================] - 336s 11s/step - loss: 1.4398 - accuracy: 0.4959 - compute_iou: 0.2243 - val_loss: 1.6624 - val_accuracy: 0.4678 - val_compute_iou: 0.2604 - lr: 1.0000e-04
Epoch 5/50
31/31 [==============================] - 335s 11s/step - loss: 1.3936 - accuracy: 0.5076 - compute_iou: 0.2325 - val_loss: 1.4457 - val_accuracy: 0.4905 - val_co

In [19]:
model.save('../../model_training/marcel')

2023-05-26 07:06:06.849549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-26 07:06:06.875550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-26 07:06:07.095237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,200,200,128]
	 [[{{node inputs}}]]
2023

INFO:tensorflow:Assets written to: ../../model_training/marcel/assets


INFO:tensorflow:Assets written to: ../../model_training/marcel/assets


In [23]:
# Evaluate the model

unet_model.evaluate_model(model, X_test_scaled, y_test)

2023-05-26 07:12:02.262486: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


1/9 [==>...........................] - ETA: 52s - loss: 1.0572 - accuracy: 0.5937 - compute_iou: 0.3172

2023-05-26 07:12:09.001319: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


2/9 [=====>........................] - ETA: 47s - loss: 1.0723 - accuracy: 0.5866 - compute_iou: 0.3136

2023-05-26 07:12:15.983286: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


...

2023-05-26 07:12:23.059431: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


4/9 [============>.................] - ETA: 34s - loss: 1.0902 - accuracy: 0.5841 - compute_iou: 0.3106

2023-05-26 07:12:30.106433: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


9/9 [==============================] - 59s 7s/step - loss: 1.0404 - accuracy: 0.6025 - compute_iou: 0.3293


ValueError: too many values to unpack (expected 2)

In [ ]:
X_train[0].shape

In [22]:
unet_model.predict(model, np.array(X_train[0]))

ValueError: in user code:

    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/marcel/.pyenv/versions/3.8.12/envs/city_sustainability/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_

In [21]:
import os
import PIL
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from PIL import ImageOps
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate

In [ ]:
file_path = '../../raw_data/OpenEarthMap_wo_xBD/paris/images/paris_1.tif'
fig = plt.figure(figsize=(3, 3))
y_original = Image.open(file_path)
plt.axis('off')
plt.imshow(y_original)
plt.show()
y_original = np.array(y_original)
y_original.shape

In [ ]:
lb = Image.open(file_path)
# Generate array for each label
# Resize each label using image_resize function
resized_label = image_resize(256,256,lb)
# Generate array for each image
numpy_array_image = np.array(resized_label)
# Encode labels
# encoded_label = to_categorical(numpy_array_label, num_classes=9)
# Add resized and encoded array to list
# label_list_array.append(encoded_label)

In [ ]:
numpy_array_image.shape

In [ ]:
from city_sustainability.models import unet_model

In [ ]:
unet

In [ ]:
lb_cat = encoded_label
lb_cat 

In [ ]:
life_quality(lb_cat)